Copyright (c) 2018 SumUp Analytics, Inc.
All Rights Reserved.
 
NOTICE:  All information contained herein is, and remains the property of SumUp
Analytics Inc. and its suppliers, if any.  The intellectual and technical
concepts contained herein are proprietary to SumUp Analytics Inc. and its
suppliers and may be covered by U.S. and Foreign Patents, patents in process,
and are protected by trade secret or copyright law.

Dissemination of this information or reproduction of this material is strictly
forbidden unless prior written permission is obtained from SumUp Analytics Inc.

# Initialization, configure API host and key, and create new API instance

In [1]:
var NucleusApi = require('nucleus_api');
var fs = require('fs'); 
var csvParser = require('csv-parse');

var host = 'UPDATE-WITH-API-HOST';
var apiKey = 'UPDATE-WITH-API-KEY';

// Create API instance
var apiClient = new NucleusApi.ApiClient(host, apiKey);
var apiInstance = new NucleusApi.NucleusApi(apiClient);

# Dataset APIs

## Append file from local drive to dataset

In [2]:
console.log("--------- Append file from local drive to dataset -----------");
var file = "quarles20181109a.pdf"; // File | 
var dataset = "dataset_test"; // String | Destination dataset where the file will be inserted.
var payload = { 
    //'metadata': "" // String | Optional json containing additional document metadata. Eg: {\"time\":\"01/01/2001\",\"author\":\"me\"}
};

var fileStream = fs.createReadStream(file)

var callback = function(error, data, response) {
    if (error) {
        console.error(error);
    } else {
        console.log(data.result + ' added to dataset ' + dataset);
        console.log('-------------------------------------------------------------');
    }
};
apiInstance.postUploadFile(fileStream, dataset, payload, callback);
console.log('-------------------------------------------------------------');


--------- Append file from local drive to dataset -----------
-------------------------------------------------------------


## Append file from URL to dataset

In [3]:
console.log("------------ Append file from URL to dataset ---------------");

var file_url = "https://www.federalreserve.gov/newsevents/speech/files/quarles20181109a.pdf"
var dataset = "dataset_test"
var payload = {
    'dataset': dataset,
    'file_url': file_url
};

var callback = function(error, data, response) {
    if (error) {
        console.error(error);
    } else {
        console.log(data.result + ' added to dataset ' + dataset);
        console.log('-------------------------------------------------------------');
    }
};
apiInstance.postUploadUrl(payload, callback);

console.log('-------------------------------------------------------------');

------------ Append file from URL to dataset ---------------
-------------------------------------------------------------


## Append json from csv to dataset

In [4]:
// This dataset will be used to test all topics and documents APIs
console.log("----------- Append json from CSV to dataset -----------------");

var csvFile = "trump-tweets-100.csv"
var dataset = "trump_tweets"

var callback = function(error, data, response) {
    if (error) {
        console.error(error);
    } else {
        //console.log('API called successfully. Returned data: ' + data.result);
    }
};

var processCsvData = function(data, rows, dataset) {
    console.log('Processing CSV data...');
    // The first row is header. Skip it.
    for (var i = 1; i < rows; i++) {
        var curRow = data[i];
        var payload = {
            'dataset': dataset,
            //'language': 'english', // optional.
            'document': {'time'   : curRow[2],
                         'title'  : curRow[4],
                         'content': curRow[1],
                         'source' : curRow[0],
                         'author' : curRow[3]}};
        
        apiInstance.postAppendJsonToDataset(payload, callback);
    }
    console.log(rows-1 + ' documents added to dataset ' + dataset);
    console.log('-------------------------------------------------------------')
}

var csvData=[];
fs.createReadStream(csvFile)
    .pipe(csvParser())
    .on('data', function(csvrow) {
        csvData.push(csvrow);        
    })
    .on('end',function() {
        processCsvData(csvData, csvData.length, dataset);
    });

console.log('-------------------------------------------------------------')

----------- Append json from CSV to dataset -----------------
-------------------------------------------------------------


## List available datasets

In [5]:
console.log('---------------- List available datasets ---------------------');

var callback = function(error, data, response) {
    if (error) {
        console.error(error);
    } else {
        var listDatasets = data.result;
        var datasetCount = listDatasets.length;
        console.log(datasetCount + ' datasets in the database:')
        for (var i = 0; i < datasetCount; i++) {
            console.log('    ' + listDatasets[i]);
        }
        console.log('-------------------------------------------------------------')
    }
};
apiInstance.getListDatasets(callback);

console.log('-------------------------------------------------------------');

---------------- List available datasets ---------------------
-------------------------------------------------------------


## Get dataset information

In [6]:
console.log('--------------- Get dataset information -------------------');
var payload = { 
    'dataset': 'dataset_test', // String | Dataset name
    'query': '', // String | Fulltext query, using mysql MATCH boolean query format.
    'metadata_selection': '', // String | json object of {\"metadata_field\":[\"selected_values\"]}
    'time_period': '' // String | Time period selection
};

var callback = function(error, data, response) {
    if (error) {
        console.error(error);
    } else {
        console.log('Information about dataset ' + dataset);
        console.log('    Language: ' + data.result.detected_language)
        console.log('    Number of documents: ' + data.result.num_documents);
        console.log('    Time range: ' + data.result.time_range[0] +
              ' to ' + data.result.time_range[1]);
        console.log('-------------------------------------------------------------');
    }
};
apiInstance.postDatasetInfo(payload, callback);

console.log('-------------------------------------------------------------');

--------------- Get dataset information -------------------
-------------------------------------------------------------


## Delete document

In [7]:
console.log('--------------------- Delete document -----------------------');
var dataset = 'dataset_test';
var docid = '1';
var payload = {
    'dataset': dataset,
    'docid': docid
}; // Deletedocumentmodel | 


var callback = function(error, data, response) {
    if (error) {
        console.error(error);
    } else {
        console.log(data.result);
        console.log('-------------------------------------------------------------');
    }
};
apiInstance.postDeleteDocument(payload, callback);

console.log('-------------------------------------------------------------');

Processing CSV data...
100 documents added to dataset trump_tweets
-------------------------------------------------------------
--------------------- Delete document -----------------------
-------------------------------------------------------------


## Delete dataset

In [8]:
console.log('--------------------- Delete dataset ------------------------');
var dataset = 'dataset_test';
var payload = {'dataset': dataset}; // Deletedatasetmodel | 

var callback = function(error, data, response) {
    if (error) {
        console.error(error);
    } else {
        console.log(dataset + ' ' + data.result);
    }
};
apiInstance.postDeleteDataset(payload, callback);


console.log('-------------------------------------------------------------');

--------------------- Delete dataset ------------------------
-------------------------------------------------------------


# Topic APIs

## Get list of topics from dataset

In [9]:
console.log('------------- Get list of topics from dataset --------------');

var payload = { 
    'dataset': 'trump_tweets', // String | Dataset name
    'query': '', // String | Fulltext query, using mysql MATCH boolean query format. Example, (\"word1\" OR \"word2\") AND (\"word3\" OR \"word4\")
    //'customStopWords': "\["real", "hillary"\]", // String | List of stop words.
    'num_topics': 8, // Number | Number of topics to be extracted from the dataset.
    'num_keywords': 8, // Number | Number of keywords per topic that is extracted from the dataset.
    'metadata_selection': '', // String | json object of {\"metadata_field\":[\"selected_values\"]}
    'time_period': '' // String | Time period selection
    //'excludedDocs': '' // String | List of document IDs that should be excluded from the analysis. Example, \"docid1, docid2, ..., docidN\" 
};

var callback = function(error, data, response) {
    if (error) {
        console.error(error);
    } else {
        for (var i = 0; i < data.result.length; i++) {
            var res = data.result[i];
            console.log('Topic', i, 'keywords:');
            console.log('    Keywords:', res.topic);
            console.log('    Keyword weights:',  res.keywords_weight);
            console.log('    Strength:', res.strength);
            var doc_topic_exposure_sel = [];  // list of non-zero doc_topic_exposure
            var doc_id_sel = [];    // list of doc ids matching doc_topic_exposure_sel
            for (var j = 0; j < res.doc_topic_exposure.length; j++) {
                var doc_topic_exp = parseFloat(res.doc_topic_exposure[j]);
                if (doc_topic_exp != 0) {
                    doc_topic_exposure_sel.push(doc_topic_exp)
                    doc_id_sel.push(res.doc_id[j])
                }
            }
            console.log('    Document IDs:', doc_id_sel)
            console.log('    Document exposures:', doc_topic_exposure_sel)

            console.log('---------------')
        }
    }
};
apiInstance.postTopicApi(payload, callback);

console.log('-------------------------------------------------------------');

------------- Get list of topics from dataset --------------
-------------------------------------------------------------


## Get topic summary

In [10]:
console.log('------------------- Get topic summary -----------------------');

var payload = { 
    'dataset': 'trump_tweets', // String | Dataset name
    'query': "", // String | Fulltext query, using mysql MATCH boolean query format. Example, (\"word1\" OR \"word2\") AND (\"word3\" OR \"word4\")
    //'custom_stop_words': "customStopWords_example", // String | List of stop words
    'num_topics': 8, // Number | Number of topics to be extracted from the dataset and summarized.
    'num_keywords': 8, // Number | Number of keywords per topic that is extracted from the dataset.
    'metadata_selection': "", // String | json object of {\"metadata_field\":[\"selected_values\"]}
    'time_period': "", // String | Time period selection
    'summary_length': 6, // Number | The maximum number of bullet points a user wants to see in each topic summary.
    'context_amount': 0, // Number | The number of sentences surrounding key summary sentences in the documents that they come from.
    'num_docs': 20 // Number | The maximum number of key documents to use for summarization.
    //'excluded_docs': "excludedDocs_example" // String | List of document IDs that should be excluded from the analysis. Example, \"docid1, docid2, ..., docidN\" 
};

var callback = function(error, data, response) {
    if (error) {
        console.error(error);
    } else {
        for (var i = 0; i < data.result.length; i++) {
            var res = data.result[i];
            console.log('Topic', i, 'summary:')
            console.log('    Keywords:', res.topic)
            for (var j = 0; j < res.summary.length; j++) {
                var summary = res.summary[j];
                console.log('    Document ID: ' + summary.sourceid);
                console.log('        Title: ' + summary.title);
                console.log('        Sentences: ' + summary.sentences);
                console.log('        Author: ' + summary.attribute.author);
                console.log('        Source: ' + summary.attribute.source);
                var docTimestamp = parseFloat(summary.attribute.time);
                var docTime = new Date(docTimestamp * 1000);
                console.log('        Time: ' + docTime.getFullYear() + '-' +
                                               docTime.getMonth() + '-' +
                                               docTime.getDate());
                console.log('---------------');
            }
        }
        console.log('-------------------------------------------------------------');
    }
};
apiInstance.postTopicSummaryApi(payload, callback);

console.log('-------------------------------------------------------------');

2 datasets in the database:
    trump_tweets
    trump_tweets_full
-------------------------------------------------------------
------------------- Get topic summary -----------------------
-------------------------------------------------------------


## Get topic sentiment

In [11]:
console.log('---------------- Get topic sentiment ------------------------');

var payload = { 
    'dataset': 'trump_tweets', // String | Dataset name
    'query': "", // String | Fulltext query, using mysql MATCH boolean query format. Example, (\"word1\" OR \"word2\") AND (\"word3\" OR \"word4\")
    //'customStopWords': "customStopWords_example", // String | List of stop words
    'num_topics': 8, // Number | Number of topics to be extracted from the dataset.
    'num_keywords': 8, // Number | Number of keywords per topic that is extracted from the dataset.
    'metadata_selection': "", // String | json object of {\"metadata_field\":[\"selected_values\"]}
    'time_period': "", // String | Time period selection
    //'excluded_docs': "excludedDocs_example", // String | List of document IDs that should be excluded from the analysis. Example, \"docid1, docid2, ..., docidN\" 
    //'custom_dict_file': "/path/to/file.txt" // File | Custom sentiment dictionary JSON file.
};

var callback = function(error, data, response) {
    if (error) {
        console.error(error);
    } else {
        for (var i = 0; i < data.result.length; i++) {
            var res = data.result[i];

            console.log('Topic', i, 'sentiment:');
            console.log('    Keywords:', res.topic);
            console.log('    Sentiment:', res.sentiment);
            console.log('    Strength:', res.strength);
            console.log('    Document IDs:', res.doc_id);
            console.log('    Document Sentiments:', res.doc_sentiment);
            console.log('    Document Scores:', res.doc_score);
    
            console.log('---------------');
        }
        console.log('-------------------------------------------------------------');
    }
};
apiInstance.postTopicSentimentApi(payload, callback);

console.log('-------------------------------------------------------------');

---------------- Get topic sentiment ------------------------
-------------------------------------------------------------


## Get topic consensus

In [12]:
console.log('---------------- Get topic consensus ------------------------')

var payload = {
    'dataset': 'trump_tweets', // String | Dataset name
    'query': "", // String | Fulltext query, using mysql MATCH boolean query format. Example, (\"word1\" OR \"word2\") AND (\"word3\" OR \"word4\")
    //'customStopWords': "", // String | List of stop words
    'num_topics': 8, // Number | Number of topics to be extracted from the dataset.
    'num_keywords': 8, // Number | Number of keywords per topic that is extracted from the dataset.
    'metadata_selection': "", // String | json object of {\"metadata_field\":[\"selected_values\"]}
    'time_period': "", // String | Time period selection
    //'excluded_docs': "excludedDocs_example", // String | List of document IDs that should be excluded from the analysis. Example, \"docid1, docid2, ..., docidN\" 
    'custom_dict_file': "" // File | Custom sentiment dictionary JSON file.
};

var callback = function(error, data, response) {
    if (error) {
        console.error(error);
    } else {
        for (var i = 0; i < data.result.length; i++) {
            var res = data.result[i];
            console.log('Topic', i, 'consensus:');
            console.log('    Keywords: ' + res.topic);
            console.log('    Consensus: ' + res.consensus);
            console.log('    Strength: ' + res.strength);
        
            console.log('---------------');
        }
    }
};
apiInstance.postTopicConsensusApi(payload, callback);

console.log('-------------------------------------------------------------')

---------------- Get topic consensus ------------------------
-------------------------------------------------------------


## Get topic historical analysis

In [13]:
console.log('------------ Get topic historical analysis ----------------');

var timePeriod = "6M"; // String | Time period selection
var updatePeriod = "d"; // String | Frequency at which the historical anlaysis is performed

var payload = {
    'dataset': 'trump_tweets', // String | Dataset name
    'time_period': '6M', // String | Time period selection. Choices: ["1M","3M","6M","12M","3Y","5Y",""]
    'update_period': 'M',  // String | Frequency at which the historical anlaysis is performed
    'query': '', // String | Fulltext query, using mysql MATCH boolean query format. Example, (\"word1\" OR \"word2\") AND (\"word3\" OR \"word4\")
    //'custom_stop_words': '' , // String | List of stop words
    'num_topics': 6, // Number | Number of topics to be extracted from the dataset.
    'num_keywords': 7, // Number | Number of keywords per topic that is extracted from the dataset.
    'metadata_selection': '', // String | json object of {\"metadata_field\":[\"selected_values\"]}
    'inc_step': 1, // Number | Number of increments of the udpate period in between two historical computations.
    //'excluded_docs': 'excludedDocs_example', // String | List of document IDs that should be excluded from the analysis. Example, \"docid1, docid2, ..., docidN\" 
    'custom_dict_file': '' // File | Custom sentiment dictionary JSON file.
};

var callback = function(error, data, response) {
    if (error) {
        console.error(error);
    } else {
        console.log('Printing historical metrics data...')
        for (var i = 0; i < data.result.length; i++) {
            var res = data.result[i];
            console.log('Topic', i, res.topic);
            console.log('    Timestamps:', res.time_stamps);
            console.log('    Strength:', res.strength);
            console.log('    Consensus:', res.consensus);
            console.log('    Sentiment:', res.sentiment);
            console.log('----------------');
        }
        console.log('-------------------------------------------------------------');
    }
};
apiInstance.postTopicHistoricalAnalysisApi(payload, callback);

console.log('-------------------------------------------------------------');

------------ Get topic historical analysis ----------------
-------------------------------------------------------------


## Get author connectivity

In [14]:
console.log('----------------- Get author connectivity -------------------');

var payload = { 
    'dataset': 'trump_tweets', // String | Dataset name,
    'target_author': "D_Trump16", // String | Name of the author to be analyzed.
    'query': "", // String | Fulltext query, using mysql MATCH boolean query format. Subject covered by the author, on which to focus the analysis of connectivity. Example, (\"word1\" OR \"word2\") AND (\"word3\" OR \"word4\")
    //'custom_stop_words': [""], // String | List of words possibly used by the target author that are considered not information-bearing.
    'time_period': '12M',  // String | Time period selection Choices: ["1M","3M","6M","12M","3Y","5Y",""]
    'metadata_selection': "", // String | json object of {\"metadata_field\":[\"selected_values\"]}
    //'excluded_docs': [""] // String | List of document IDs that should be excluded from the analysis. Example, \"docid1, docid2, ..., docidN\" 
};

var callback = function(error, data, response) {
    if (error) {
        console.error(error);
    } else {
        var res = data.result;
        console.log('Mainstream connections:')
        for (var i = 0; i < res.mainstream_connection.length; i++) {
            var mc = res.mainstream_connection[i];
            console.log('    Topic:', mc.topic);
            console.log('    Authors:', mc.authors);
        }
        
        console.log('Niche connections:')
        for (var i = 0; i < res.niche_connection.length; i++) {
            var nc = res.niche_connection[i];
            console.log('    Topic:', nc.topic);
            console.log('    Authors:', nc.authors);
        }
        console.log('-------------------------------------------------------------');
  }
};
apiInstance.postAuthorConnectivityApi(payload, callback);

console.log('-------------------------------------------------------------');

----------------- Get author connectivity -------------------
-------------------------------------------------------------


## Get topic delta

In [15]:
console.log('------------------- Get topic deltas -----------------------');

var payload = {
    'dataset': 'trump_tweets',                               
    'query': '', // String | Fulltext query, using mysql MATCH boolean query format. Example, (\"word1\" OR \"word2\") AND (\"word3\" OR \"word4\")
    //'custom_stop_words': "customStopWords_example", // String | List of stop words.
    'num_topics': 8, // Number | Number of topics to be extracted from the dataset.
    'num_keywords': 8, // Number | Number of keywords per topic that is extracted from the dataset.
    'period_0_start': '2018-08-12 00:00:00',
    'period_0_end': '2018-08-15 13:00:00',
    'period_1_start': '2018-08-16 00:00:00',
    'period_1_end': '2018-08-19 00:00:00',
    'metadata_selection': '', // String | json object of {\"metadata_field\":[\"selected_values\"]}
    //'excluded_docs': 'excludedDocs_example' // String | List of document IDs that should be excluded from the analysis. Example, \"docid1, docid2, ..., docidN\" 
};

var callback = function(error, data, response) {
    if (error) {
        console.error(error);
    } else {
        for (var i = 0; i < data.result.length; i++) {
            var res = data.result[i];
            console.log('Topic', i, 'changes in exposure:');
            console.log('    Keywords:', res.topic);
            console.log('    Document ID:', res.doc_id_t0, res.doc_id_t1);
            console.log('    Per Source Change in Exposure:', res.doc_topic_exposure_delta);
            console.log('---------------');
        }
    }
};
apiInstance.postTopicDeltaApi(payload, callback);

console.log('-------------------------------------------------------------');
//

------------------- Get topic deltas -----------------------
-------------------------------------------------------------


# Document APIs

## Get document information without content

In [16]:
console.log('----------------- Display document information --------------------');

var payload = {
    'dataset': 'trump_tweets', // String | Dataset name,
    'docTitles': "[ \"D_Trump2018_8_18_1_47\" ]", // String | The title of the documents on which info is requested. Example: [ \"title 1\", \"title 2\" ] 
    'docIds': "[ \"11\", \"12\", \"13\" ]", // String | The docid of the documents on which info is requested. Example:[ \"docid1, docid2\" ]
    'metadata_selection': ''
};

var callback = function(error, data, response) {
    if (error) {
        console.error(error);
    } else {
        for (var i = 0; i < data.result.length; i++) {
            var res = data.result[i];
            console.log('Document ID:', res.sourceid);
            console.log('    Title:', res.title);
            console.log('    Author:', res.attribute.author);
            console.log('    Source:', res.attribute.source);
            console.log('    Time:', res.attribute.time);

            console.log('---------------')
        }
    }
};
apiInstance.postDocInfo(payload, callback);

console.log('-------------------------------------------------------------');

----------------- Display document information --------------------
-------------------------------------------------------------


## Display document details

In [17]:
console.log('----------------- Display document details --------------------');

var payload = { 
    'dataset': 'trump_tweets', // String | Dataset name,
    'docTitles': "[ \"D_Trump2018_8_18_1_47\" ]", // String | The title of the documents on which info is requested. Example: [ \"title 1\", \"title 2\" ] 
    'docIds': "[ \"11\", \"12\", \"13\" ]" // String | The docid of the documents on which info is requested. Example:[ \"docid1, docid2\" ]
};

var callback = function(error, data, response) {
    if (error) {
        console.error(error);
    } else {
        for (var i = 0; i < data.result.length; i++) {
            var res = data.result[i];
            console.log('Document ID:' + res.sourceid);
            console.log('    Title:' + res.title);
            console.log('    Author:' + res.attribute.author);
            console.log('    Source:' + res.attribute.source);
            console.log('    Time:', res.attribute.time);
            console.log('    Content', res.content);

            console.log('---------------')
        }
        console.log('-------------------------------------------------------------')
    }
};
apiInstance.postDocDisplay(payload, callback);

console.log('-------------------------------------------------------------')

----------------- Display document details --------------------
-------------------------------------------------------------


## Get document recommendations

In [18]:
console.log('------------- Get document recommendations -----------------')

var payload = { 
    'dataset': 'trump_tweets', // String | Dataset name,
    'query': "", // String | Fulltext query, using mysql MATCH boolean query format. Example, (\"word1\" OR \"word2\") AND (\"word3\" OR \"word4\")
    //'customStopWords': "customStopWords_example", // String | List of stop words.
    'num_topics': 8, // Number | Number of topics to be extracted from the dataset.
    'num_keywords': 8, // Number | Number of keywords per topic that is extracted from the dataset.
    'num_docs': 20, // Number | Number of desired recommended docs per topic.
    //'metadata_selection': "metadataSelection_example", // String | json object of {\"metadata_field\":[\"selected_values\"]}
    //'time_period': "timePeriod_example", // String | Time period selection
    //'excluded_docs': "excludedDocs_example" // String | List of document IDs that should be excluded from the analysis. Example, \"docid1, docid2, ..., docidN\" 
};

var callback = function(error, data, response) {
    if (error) {
        console.error(error);
    } else {
        for (var i = 0; i < data.result.length; i++) {
            var res = data.result[i];
            console.log('Document recommendations for topic', i, ':')
            console.log('    Keywords:', res.topic);

            for (var j = 0; j < res.recommendations.length; j++) {
                var doc = res.recommendations[j];
                console.log('    Recommendation', j, ':')
                console.log('        Document ID:', doc.sourceid)
                console.log('        Title:', doc.title)
                console.log('        Author:', doc.attribute.author)
                console.log('        Source:', doc.attribute.source)
                console.log('        Time:', doc.attribute.time)
            }
    
            console.log('---------------')
        }
    }
};
apiInstance.postDocRecommendApi(payload, callback);

console.log('-------------------------------------------------------------')

------------- Get document recommendations -----------------
-------------------------------------------------------------


## Get document summary

In [19]:
console.log('------------------ Get document summary  --------------------')

var dataset = "trump_tweets"; // String | Dataset name.
var doc_title = "D_Trump2018_8_15_15_4"; // String | The title of the document to be summarized.

var payload = { 
    'dataset': 'trump_tweets', // String | Dataset name,
    'doc_title': 'D_Trump2018_8_15_15_4', // String | The title of the document to be summarized.
    //'custom_stop_words': "customStopWords_example", // String | List of stop words
    'summary_length': 6, // Number | The maximum number of bullet points a user wants to see in the document summary.
    'context_amount': 0 // Number | The number of sentences surrounding key summary sentences in the documents that they come from.
};

var callback = function(error, data, response) {
    if (error) {
        console.error(error);
    } else {
        console.log('Document Summary');
        console.log('    ID:', data.result.summary.sourceid);
        console.log('    Title:', data.result.doc_title);
        console.log('    Summary:', data.result.summary.sentences);
    }
};
apiInstance.postDocSummaryApi(payload, callback);

console.log('-------------------------------------------------------------')


------------------ Get document summary  --------------------
-------------------------------------------------------------


{ Error: Bad Request
    at Request.callback (/Users/jimw/github/sumup/nucleus-api/nucleus/rest_api/swagger/javascript/js-api/node_modules/superagent/lib/node/index.js:688:13)
    at /Users/jimw/github/sumup/nucleus-api/nucleus/rest_api/swagger/javascript/js-api/node_modules/superagent/lib/node/index.js:891:18
    at IncomingMessage.<anonymous> (/Users/jimw/github/sumup/nucleus-api/nucleus/rest_api/swagger/javascript/js-api/node_modules/superagent/lib/node/parsers/json.js:17:7)
    at IncomingMessage.emit (events.js:187:15)
    at endReadableNT (_stream_readable.js:1090:12)
    at process._tickCallback (internal/process/next_tick.js:63:19)
  status: 400,
  response:
   Response {
     _events: {},
     _eventsCount: 0,
     _maxListeners: undefined,
     res:
      IncomingMessage {
        _readableState: [ReadableState],
        readable: false,
        _events: [Object],
        _eventsCount: 4,
        _maxListeners: undefined,
        socket: [Socket],
        connection: [Socket]

Information about dataset trump_tweets
    Language: undefined
    Number of documents: undefined


{ TypeError: Cannot read property '0' of undefined
    at callback (evalmachine.<anonymous>:16:64)
    at cb (/Users/jimw/github/sumup/nucleus-api/nucleus/rest_api/swagger/javascript/js-api/src/api/NucleusApi.js:98:29)
    at /Users/jimw/github/sumup/nucleus-api/nucleus/rest_api/swagger/javascript/js-api/src/ApiClient.js:492:9
    at Request.callback (/Users/jimw/github/sumup/nucleus-api/nucleus/rest_api/swagger/javascript/js-api/node_modules/superagent/lib/node/index.js:681:16)
    at /Users/jimw/github/sumup/nucleus-api/nucleus/rest_api/swagger/javascript/js-api/node_modules/superagent/lib/node/index.js:891:18
    at IncomingMessage.<anonymous> (/Users/jimw/github/sumup/nucleus-api/nucleus/rest_api/swagger/javascript/js-api/node_modules/superagent/lib/node/parsers/json.js:17:7)
    at IncomingMessage.emit (events.js:187:15)
    at endReadableNT (_stream_readable.js:1090:12)
    at process._tickCallback (internal/process/next_tick.js:63:19)
  response:
   Response {
     _events: {},


Information about dataset trump_tweets
    Language: undefined
    Number of documents: undefined


...

    Content undefined
---------------
Document ID:5972
    Title:D_Trump2018_8_14_11_31
    Author:D_Trump72
    Source:Twitter for iPhone
    Time: 1534246260
    Content undefined
---------------
Document ID:5973
    Title:D_Trump2018_8_14_11_55
    Author:D_Trump70
    Source:Twitter for iPhone
    Time: 1534247700
    Content undefined
---------------
Document ID:5975
    Title:D_Trump2018_8_14_11_13
    Author:D_Trump74
    Source:Twitter for iPhone
    Time: 1534245180
    Content undefined
---------------
Document ID:5976
    Title:D_Trump2018_8_14_1_57
    Author:D_Trump77
    Source:Twitter for iPhone
    Time: 1534211820
    Content undefined
---------------
Document ID:5977
    Title:D_Trump2018_8_14_10_59
    Author:D_Trump76
    Source:Twitter for iPhone
    Time: 1534244340
    Content undefined
---------------
Document ID:5978
    Title:D_Trump2018_8_14_0_11
    Author:D_Trump81
    Source:Twitter for iPhone
    Time: 1534205460
    Content undefined
---------------
Docu

    Timestamps: [ '2018-08-10 17:59',
  '2018-08-10 18:59',
  '2018-08-10 19:59',
  '2018-08-10 20:59',
  '2018-08-10 21:59',
  '2018-08-10 22:59',
  '2018-08-10 23:59',
  '2018-08-11 00:59',
  '2018-08-11 01:59',
  '2018-08-11 02:59',
  '2018-08-11 03:59',
  '2018-08-11 04:59',
  '2018-08-11 05:59',
  '2018-08-11 06:59',
  '2018-08-11 07:59',
  '2018-08-11 08:59',
  '2018-08-11 09:59',
  '2018-08-11 10:59',
  '2018-08-11 11:59',
  '2018-08-11 12:59',
  '2018-08-11 13:59',
  '2018-08-11 14:59',
  '2018-08-11 15:59',
  '2018-08-11 16:59',
  '2018-08-11 17:59',
  '2018-08-11 18:59',
  '2018-08-11 19:59',
  '2018-08-11 20:59',
  '2018-08-11 21:59',
  '2018-08-11 22:59',
  '2018-08-11 23:59',
  '2018-08-12 00:59',
  '2018-08-12 01:59',
  '2018-08-12 02:59',
  '2018-08-12 03:59',
  '2018-08-12 04:59',
  '2018-08-12 05:59',
  '2018-08-12 06:59',
  '2018-08-12 07:59',
  '2018-08-12 08:59',
  '2018-08-12 09:59',
  '2018-08-12 10:59',
  '2018-08-12 11:59',
  '2018-08-12 12:59',
  '2018-08-12 13

    Consensus: [ 'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  '1.0',
  '1.0',
  '1.0',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  'nan',
  'nan',
  'nan',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  'nan',
  'nan',
  '1.0',
  '1.0',
  ... 4221 more items ]
    Sentiment: [ 'nan',
  'nan',
  'nan',
  'nan',
  'nan',
 

    Strength: [ 'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  '0.5',
  '0.3333333333333333',
  '0.4472684621845538',
  '0.4472684621845538',
  '0.4472684621845538',
  '0.4472684621845538',
  '0.40459828287733235',
  '0.6795383205016439'

----------------
Topic 5 donald trump;witch hunt;frame donald;hillary clinton;clinton frame;jarrett foxandfriends;gregg jarrett
    Timestamps: [ '2018-08-10 17:59',
  '2018-08-10 18:59',
  '2018-08-10 19:59',
  '2018-08-10 20:59',
  '2018-08-10 21:59',
  '2018-08-10 22:59',
  '2018-08-10 23:59',
  '2018-08-11 00:59',
  '2018-08-11 01:59',
  '2018-08-11 02:59',
  '2018-08-11 03:59',
  '2018-08-11 04:59',
  '2018-08-11 05:59',
  '2018-08-11 06:59',
  '2018-08-11 07:59',
  '2018-08-11 08:59',
  '2018-08-11 09:59',
  '2018-08-11 10:59',
  '2018-08-11 11:59',
  '2018-08-11 12:59',
  '2018-08-11 13:59',
  '2018-08-11 14:59',
  '2018-08-11 15:59',
  '2018-08-11 16:59',
  '2018-08-11 17:59',
  '2018-08-11 18:59',
  '2018-08-11 19:59',
  '2018-08-11 20:59',
  '2018-08-11 21:59',
  '2018-08-11 22:59',
  '2018-08-11 23:59',
  '2018-08-12 00:59',
  '2018-08-12 01:59',
  '2018-08-12 02:59',
  '2018-08-12 03:59',
  '2018-08-12 04:59',
  '2018-08-12 05:59',
  '2018-08-12 06:59',
  '2018-08-12 07:59'